# Determining FIB Wedge Angle Using t/Lamda Measurements

This is a personal workbook used to process my own data but has been made public for the use of others. If you find any errors or need further help using this workbook please contact k8macarthur@gmail.com. 

## Authors

Katherine E. MacArthur - Originally written for EMC2016, 28.08.2016

## Requirements

This code was written to work Hyperspy version 1.1 or later.

## 1. Importing Hyperspy and Libraries

Begin by importing the Hyperspy Function Library

In [1]:
import matplotlib
matplotlib.rcParams["backend"] = 'Qt4Agg'
import hyperspy.api as hs
import numpy as np
%matplotlib qt4

unbcf_fast library is not present...
Falling back to slow python only backend.


In [3]:
%matplotlib inline

## 2. Importing and Aligning Data

Opens a load window. Use hs.load('filename') if filename is known specifically.

In [2]:
wedge = hs.load()

In [3]:
wedge.set_signal_type('EELS')
wedge.metadata.General.title = 'Original Data Wedge'
wedge.set_microscope_parameters(beam_energy = 200)
wedge

<EELSSpectrum, title: Original Data Wedge, dimensions: (346, 8|2048)>

Check current metadata.

In [4]:
wedge.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       ├── aperture_size = 5.0
│       │       ├── collection_angle = 15.151515007019043
│       │       ├── frame_number = 625
│       │       └── spectrometer = Enfinium ER
│       ├── beam_energy = 200
│       └── convergence_angle = 25.0
├── General
│   ├── date = 2016-09-14
│   ├── original_filename = EELS Spectrum Image.dm3
│   ├── time = 12:04:37
│   └── title = Original Data Wedge
└── Signal
    ├── binned = True
    └── signal_type = EELS

In [5]:
wedge.plot()
wedge.axes_manager

<Axes manager, axes: (346, 8|2048)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    346 |      0 |      -0 |       1 |        
               y |      8 |      0 |      -0 |       1 |        
---------------- | ------ | ------ | ------- | ------- | ------ 
     Energy loss |   2048 |        |     -50 |    0.25 |     eV

Adjust the metadata for all the sample parameters.

In [6]:
wedge.metadata.Acquisition_instrument.TEM.dwell_time = 1
wedge.axes_manager[0].scale = 55.7
wedge.axes_manager[1].scale = 55.7

Extract t over lambda using estimate_thickness(point), where point is the first minimum point after the zero loss peak.

In [7]:
wedge.align_zero_loss_peak()


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-6.168
std:	3.134

min:	-11.750
Q1:	-8.750
median:	-6.250
Q3:	-3.750
max:	4.500


100%|██████████████████████████████████████████████████████████████████████████████| 2768/2768 [03:45<00:00, 11.30it/s]


In [8]:
wedge.isig[:].spikes_removal_tool()

## 4. Extracting and Estimating Thickness Information

In [9]:
s = wedge.estimate_elastic_scattering_threshold()

In [10]:
wedge.plot()

In [11]:
thickness = wedge.estimate_thickness(s)
thickness.plot()

100%|██████████████████████████████████████████████████████████████████████████████| 2768/2768 [04:30<00:00, 10.22it/s]


In order to convert t/lambda maps to thickness maps lambda has to be estimated. Here I normally use the calculator by Nestor Zaluzec. tpm.amc.anl.gov/NJZTools/NTLambdaV2.html

In [12]:
lamda = 70.21
thickness = thickness*lamda

## 5. Converting from the Intensity Map to a Line Profile

This assumes that the line profile is parallel to the x-axis in the image.

In [35]:
width = thickness.axes_manager[0].scale
line_scan = np.zeros(thickness.data.shape[1])
line_scan.astype(float)
for row in range(thickness.data.shape[0]):
    for col in range(thickness.data.shape[1]):
        line_scan[col] = line_scan[col] + thickness.data[row, col]
    
line_scan = line_scan/thickness.data.shape[0]

x_data = np.zeros(thickness.data.shape[1])
x_data.astype(float)
for i in range(thickness.data.shape[1]):
    x_data[i] = i*width

In [36]:
line_scan = line_scan[::-1] 

In [38]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6), dpi=80)
plt.plot(x_data, line_scan)
plt.show()

An iterative cropping method should be applied to select only linear area of the spectrum.

In [37]:
line_scan = line_scan[40:-170]
x_data = x_data[40:-170]

Determining the gradient measurement from the linear profile extracted from the t/lambda map.

In [39]:
from scipy.optimize import curve_fit
def func(x, m, c):
    return m*x + c
popt, pcov = curve_fit(func, x_data, line_scan)
m = popt[0]
print('m=', m)
perr = np.sqrt(np.diag(pcov))
m_err = perr[0] #the error in the gradient measurment due to noise.

m= 0.00482308238563


In [40]:
m_err/m

0.0090307761745501685

Determining the angle from the gradient measurement.

In [41]:
import math
theta = math.degrees(math.atan(m))
theta

0.2763401222011879

Save the processed data cube for later.

In [ ]:
wedge.save('Pt01EELS-processed')